<a href="https://colab.research.google.com/github/eyaler/Qwen-Image-Edit-Plus-Lightning-Feedback-Loop/blob/main/Qwen_Image_Edit_Plus_Lightning_Feedback_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



based on: https://github.com/radames/nano-banana-loop

requires: A100 High-RAM


In [ ]:
#@title Install

!pip install git+https://github.com/huggingface/diffusers

import torch
import math
from diffusers import FlowMatchEulerDiscreteScheduler, QwenImageEditPlusPipeline
from diffusers.models import QwenImageTransformer2DModel
from diffusers.utils import load_image
from google.colab import files
from IPython.display import Video
import re
from time import perf_counter


lora = '8steps'  #@param ['4steps', '8steps', 'None']

torch_dtype = torch.bfloat16
model_name = 'Qwen/Qwen-Image-Edit-2509'
steps = 50
cfg = 4
negative_prompt = ' '

if lora != 'None':
  lora_repo = 'lightx2v/Qwen-Image-Lightning'
  lora_path = f'Qwen-Image-Edit-2509/Qwen-Image-Edit-2509-Lightning-{lora}-V1.0-bf16.safetensors'
  steps = int(re.search('\\d+(?=steps)', lora_path).group())
  cfg = 1
  negative_prompt = None

  model = QwenImageTransformer2DModel.from_pretrained(model_name, subfolder='transformer', torch_dtype=torch_dtype)

  scheduler_config = {
      "base_image_seq_len": 256,
      "base_shift": math.log(3),  # We use shift=3 in distillation
      "invert_sigmas": False,
      "max_image_seq_len": 8192,
      "max_shift": math.log(3),  # We use shift=3 in distillation
      "num_train_timesteps": 1000,
      "shift": 1.0,
      "shift_terminal": None,  # set shift_terminal to None
      "stochastic_sampling": False,
      "time_shift_type": "exponential",
      "use_beta_sigmas": False,
      "use_dynamic_shifting": True,
      "use_exponential_sigmas": False,
      "use_karras_sigmas": False,
  }
  scheduler = FlowMatchEulerDiscreteScheduler.from_config(scheduler_config)
  pipe = QwenImageEditPlusPipeline.from_pretrained(model_name, transformer=model, scheduler=scheduler, torch_dtype=torch_dtype)
  pipe.load_lora_weights(lora_repo, weight_name=lora_path)
else:
  pipe = QwenImageEditPlusPipeline.from_pretrained(model_name, torch_dtype=torch_dtype)

pipe = pipe.to('cuda')
pipe.set_progress_bar_config(disable=True)

In [ ]:
#@title Upload image

uploaded = files.upload('/content/sample_data')
if uploaded:
  img_filename = list(uploaded)[0]
  init_image = load_image(img_filename)
  print(img_filename)
init_image

In [ ]:
#@title Generate

task_or_prompt = 'next'  #@param ["next","prev","motion","background","up","down","left","right","rotate-left","rotate-right","zoom-in","zoom-out","future","past","funny","serious","dramatic","peaceful","vintage","futuristic","nature","urban","minimalist","crowded","empty"] {"allow-input":true}
#@markdown (you can also write a custom prompt)
seed = 42  #@param {type: 'integer'}
duration_secs = 5  #@param {type: 'number'}
fps = 8  #@param {type: 'integer'}

prompts = {
    "next": "Show the next scene one second later",
    "prev": "Show the previous scene one second before",
    "motion": "Apply subtle motions to create the next animation frame",
    "background": "Gently change the background of the moving object",
    "up": "Gently pan the camera up, extending the image.",
    "down": "Gently pan the camera down, extending the image.",
    "left": "Gently pan the camera left, extending the image.",
    "right": "Gently pan the camera right, extending the image.",
    "rotate-left": "Gently rotate the camera counter-clockwise, extending the borders to fit the new perspective.",
    "rotate-right": "Gently rotate the camera clockwise, extending the borders to fit the new perspective.",
    "zoom-in": "Gently zoom in on the center of the image, maintaining focus and detail.",
    "zoom-out": "Gently zoom out from the image, revealing more of the surrounding scene.",
    "future": "Show this scene one second in the future",
    "past": "Show this scene one second in the past",
    "funny": "Subtly alter this image by replacing one or two details with something unexpected and funny.",
    "serious": "Subtly alter this image by replacing one or two details with something more serious, meaningful, or thought-provoking.",
    "dramatic": "Subtly enhance the drama and intensity of this scene. Adjust lighting to be more cinematic, deepen shadows, or add atmospheric elements like mist or dramatic sky. Keep changes photorealistic and well-integrated.",
    "peaceful": "Transform this scene to be more peaceful and serene. Soften harsh elements, add calming details like gentle lighting or natural elements. Keep changes subtle and photorealistic.",
    "vintage": "Apply a subtle vintage aesthetic to this image. Add slight film grain, adjust colors to warmer or cooler vintage tones, and create a nostalgic atmosphere while maintaining photorealism.",
    "futuristic": "Subtly modernize or add futuristic elements to this scene. Replace one or two objects with sleek, high-tech alternatives. Keep changes minimal, well-integrated, and photorealistic.",
    "nature": "Subtly introduce natural elements into this scene. Add plants, natural lighting, or organic textures. Keep changes small and seamlessly integrated with photorealistic quality.",
    "urban": "Subtly add urban elements to this scene. Introduce architectural details, city textures, or modern infrastructure. Keep changes minimal and photorealistic.",
    "minimalist": "Simplify this scene with minimalist aesthetics. Remove or tone down one or two distracting elements, create cleaner compositions, and emphasize negative space. Keep it photorealistic.",
    "crowded": "Subtly add more people or objects to make this scene feel more populated or busy. Keep additions natural, well-integrated, and photorealistic.",
    "empty": "Subtly remove one or two people or objects to make this scene feel more spacious or isolated. Keep the result natural and photorealistic.",
}

if task_or_prompt in prompts:
  prompt = prompts[task_or_prompt]
  output_filename = task_or_prompt + '.mp4'
else:
  prompt = task_or_prompt
  output_filename = 'custom.mp4'
print(prompt)

total_frames = max(2, round(duration_secs * fps))
digits = len(str(total_frames))

!rm -rf /content/output
!mkdir /content/output
image = init_image
image.save(f'/content/output/{"0".zfill(digits)}.jpg')

gen = torch.Generator('cuda').manual_seed(seed)

start_time = perf_counter()

for i in range(1, total_frames):
  image = pipe(image=image, prompt=prompt, negative_prompt=negative_prompt, true_cfg_scale=cfg, num_inference_steps=steps, generator=gen).images[0]
  image.save(f'/content/output/{str(i).zfill(digits)}.jpg')

dt = perf_counter() - start_time
print(f'took: {dt:.0f} secs, {dt/duration_secs:.1f} sec/sec, {dt/(total_frames - 1):.1f} sec/frame')

!ffmpeg -framerate $fps -i "/content/output/%0{digits}d.jpg" -pix_fmt yuv420p -movflags +faststart -y $output_filename -hide_banner -loglevel error
Video(output_filename, embed=True, html_attributes='autoplay controls loop')


In [ ]:
#@title Download
files.download(output_filename)